<a href="https://colab.research.google.com/github/mzohaibnasir/GenAI/blob/main/05_LangchainProject_MCQGenerator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install openai langchain streamlit python-dotenv PyPDF2

In [2]:
from google.colab import userdata

OPENAIAPIKEY = userdata.get("OPENAIAPIKEY")

In [4]:
from openai import OpenAI

client = OpenAI(
    api_key=OPENAIAPIKEY
    )